In [17]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F  # a module in PyTorch that contains functions like activation functions, loss functions, and more.

In [18]:
#  this line of code sets up a pipeline that first converts images to the tensor format suitable for neural networks and then normalizes their pixel values to help with the efficient training of the network.
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))] )

In [19]:
print(transform)

Compose(
    ToTensor()
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
)


In [20]:
# CIFAR-10 dataset contains 60,000 color images of 32x32 pixels, evenly divided into 10 classes (like cars, birds, cats, etc.), with 6,000 images per class. The dataset is split into 50,000 training images and 10,000 test images.

# Load the training portion of the CIFAR-10 dataset, download it to the specified folder if it's not there already, and apply the specified preprocessing steps to each image in the dataset.
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

Files already downloaded and verified


In [21]:
#  this line of code sets up an efficient way to iterate through the training data, providing batches of four images at a time, shuffled for each epoch to improve learning, and using two parallel workers to ensure that data is always ready for the model to train on, minimizing downtime. 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

In [22]:
# for the test set:
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified


In [23]:
# The Net class represents a specific architecture for a Convolutional Neural Network (CNN).
# 3 channels for RGB color images.
# 6 filters apply, 6 different feature maps, where each feature map is essentially a 2D array that represents some features detected in the input.
# 5 the size of the kernel.
# The second convolutional layer takes the 6 channels produced by conv1 as input and produces 16 output channels, also with a 5x5 kernel.
# A max pooling layer with a 2x2 window and stride of 2. 
# fc1, fc2, fc3: 

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)  # The output of the last pooling layer is flattened to transform it into a vector suitable for input into the fully connected layers.
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [24]:
net = Net()